In [1]:
import tensorflow
import stellargraph
import numpy
import matplotlib
import os.path
import pandas
import datetime
import re

Please notice, to open jupyter in custom directory use:
>jupyter notebook --notebook-dir=[your_directory]

Expecting file hierarchy:
    <ul>root/
        <li>Antarctica_research_2020/
            <ul>.git/</ul>
            <ul>.gitignore</ul>
            <ul>readme.md</ul>
            <ul>notebook.ipynb</ul></li>
        <li>data/
            <ul>Sesja 1.xlsx</ul>
            <ul>.</ul>
            <ul>.</ul>
            <ul>Sesja 26.xlsx</ul></li>
    </ul>

In [2]:
# Let me write in advantage examined peoples' occupations:
ppl_professions = {1: "to be filled up",
                  2: "to be filled up",
                  3: "to be filled up",
                  4: "to be filled up",
                  5: "to be filled up",
                  6: "to be filled up",   # UWAGA - nr 6 przybył na stację 25.04.79r
                  7: "to be filled up",
                  8: "to be filled up",
                  9: "to be filled up",
                  10: "to be filled up",
                  11: "to be filled up",
                  12: "to be filled up",
                  13: "to be filled up",
                  14: "to be filled up",
                  15: "to be filled up",
                  16: "to be filled up",
                  17: "to be filled up",
                  18: "to be filled up",
                  19: "to be filled up",
                  20: "to be filled up",
                  21: "to be filled up"
                  }

In [3]:
data_path = "../data/"
file_list = [name for name in os.listdir(data_path)]
print(file_list)

['Sesja 1.xlsx', 'Sesja 10.xlsx', 'Sesja 11.xlsx', 'Sesja 12.xlsx', 'Sesja 13.xlsx', 'Sesja 14.xlsx', 'Sesja 15.xlsx', 'Sesja 16.xlsx', 'Sesja 17.xlsx', 'Sesja 18.xlsx', 'Sesja 19.xlsx', 'Sesja 2.xlsx', 'Sesja 20.xlsx', 'Sesja 21.xlsx', 'Sesja 22.xlsx', 'Sesja 23.xlsx', 'Sesja 24.xlsx', 'Sesja 25.xlsx', 'Sesja 26.xlsx', 'Sesja 3.xlsx', 'Sesja 4.xlsx', 'Sesja 5.xlsx', 'Sesja 6.xlsx', 'Sesja 7.xlsx', 'Sesja 8.xlsx', 'Sesja 9.xlsx']


In [4]:
# Sessions order is important, let me get sorted file info
sorted_filenames = dict()
for name in file_list:
    list_of_digits = list(map(int, filter(str.isdigit, name)))
    sorted_filenames[name] = list_of_digits[0] if len(list_of_digits) == 1 else list_of_digits[1] + list_of_digits[0]*10
sorted_filenames = sorted(sorted_filenames.items(), key=lambda el: el[1])
sorted_filenames = {val: key for key, val in sorted_filenames}
print(sorted_filenames)

{1: 'Sesja 1.xlsx', 2: 'Sesja 2.xlsx', 3: 'Sesja 3.xlsx', 4: 'Sesja 4.xlsx', 5: 'Sesja 5.xlsx', 6: 'Sesja 6.xlsx', 7: 'Sesja 7.xlsx', 8: 'Sesja 8.xlsx', 9: 'Sesja 9.xlsx', 10: 'Sesja 10.xlsx', 11: 'Sesja 11.xlsx', 12: 'Sesja 12.xlsx', 13: 'Sesja 13.xlsx', 14: 'Sesja 14.xlsx', 15: 'Sesja 15.xlsx', 16: 'Sesja 16.xlsx', 17: 'Sesja 17.xlsx', 18: 'Sesja 18.xlsx', 19: 'Sesja 19.xlsx', 20: 'Sesja 20.xlsx', 21: 'Sesja 21.xlsx', 22: 'Sesja 22.xlsx', 23: 'Sesja 23.xlsx', 24: 'Sesja 24.xlsx', 25: 'Sesja 25.xlsx', 26: 'Sesja 26.xlsx'}


In [5]:
# Loading data
data_dict = dict()
for key, val in sorted_filenames.items():
    data_dict[key] = pandas.read_excel(data_path + val, header=None, dtype=object)

In [15]:
rand_int = int(numpy.random.rand() * 26)
data_dict[rand_int].head(4)
# An example:

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,NaN,Pozycja osoby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N = 21,NaN,NaN,1979-07-01 00:00:00,NaN,NaN,NaN,ZIM14,NaN,NaN
1,Nr O.B.,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
2,1,2719,2815,2829,2826,2626,2824,2426,2917,2919,...,2417,2419,2421,2423,2914,2714,2724,2522,2516,2425
3,2,2912,2201,2311,2322,1818,1710,3121,1115,1119,...,3437,2133,1434,2006,1124,0733,1724,3629,0517,0524


The target format for data is a stellargraph graph for each session (session -> data recording occuring every two week). But before that, let me read the data into intermediate objects:

In [37]:
class Session(object):
    def __init__(self, raw_session_data):
        self.date = self.find_date(raw_session_data)
        self.relations = self.get_relations(raw_session_data) # dictionary of lists of tuples - column_number : [(x1, y1), (x2, y2), ...]
    
    def get_relations(self, raw_session_data) -> dict:
        rels = dict()
        col_no = len(ppl_professions)
        rels = {x: [] for x in range(1, col_no + 1)}
        for person in range(2, col_no + 1):
            for entry in range(1, col_no):
                string_value = raw_session_data.iloc[person, entry]
                subject_no = raw_session_data.iloc[person, 0]
                rels[subject_no].append(string_value)
                
        return rels
    
    def find_date(self, raw_session_data) -> str:
        for key, col in raw_session_data.iteritems():
            for val in col:
                regex = re.search(r"\d{1,2}.\d{1,2}.\d{4}|\d{4}.\d{1,2}.\d{1,2}", str(val))
                if regex:
                    return regex.group()
                
    def __str__(self):
        return "Session from " + self.date + " example entry: " + str(self.relations[1])

In [38]:
class Person(object):
    def __init__(self, id, date, relations: list):
        pass

In [39]:
test_session = Session(data_dict[1])
print(test_session)

Session from 1978-12-29 example entry: [2420, 2518, 1232, 2123, 2430, 1930, 2625, 2921, 1218, 2422, 1716, 2311, 2812, 1526, 2520, 2120, 1224, 2012, 1822, 2915]
